In [1]:
import numpy as np

import os, pickle, time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 이런식의 코드를 막 그냥 작성해버리면 나중에 알아보기 어렵습니다.

- <span style = 'font-size:1.2em;line-height:1.5em'>게다가, Jupyter Notebook형식의 .ipynb로 작성하면 나중에 이 코드를 다시 써먹기가 어렵습니다.</span>

- <span style = 'font-size:1.2em;line-height:1.5em'>좀 헷갈려도 코드를 모듈화하고 스크립트 파일인 .py파일로 변환해줄 필요가 있습니다.</span>

# 필요한 코드들을 모듈화 해보겠습니다.

## 1. 네트워크 구조 & Input Flow 작성하기
- <span style = 'font-size:1.2em;line-height:1.5em'>Input(28*28) -> 1st convolution(+relu) + MaxPooling -> 2nd convolution(+relu) + Maxpooling -> (Flatten) -> 1st FFNN(+relu) -> 2nd FFNN </span>

    - <span style = 'font-size:1.1em;line-height:1.5em'><b>1st conv</b></span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>in_channel=1, out_channel=64: </b>1개 channel을 갖는 image를 받아 64개의 channel을 생성</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>kernel_size=(3,3): </b>채널의 각 위치의 값을 계산하는 kernel의 크기는 height 3, width 3. 이러한 kernel이 1*64개만큼 생성됨</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>stride=(1,1), padding=1: </b>kernel의 이동은 세로로 1, 가로로 1만큼. conv층에 들어오는 데이터에 padding을 양옆에 1만큼 해줌</span>
        
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>1st MaxPooling</b></span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>kernel_size=(2,2): </b>kernel의 크기는 height 2, width 2. 이러한 kernel이 conv layer의 out_channel개만큼 생성됨</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>stride=(2,2): </b>kernel의 이동은 세로로 2, 가로로 2만큼.</span>
        
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>2st conv</b></span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>in_channel=64, out_channel=`18: </b>64개 channel을 갖는 1st MaxPooling 층의 output값을 128개의 channel을 생성</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>kernel_size=(3,3): </b>채널의 각 위치의 값을 계산하는 kernel의 크기는 height 3, width 3. 이러한 kernel이 64*128개만큼 생성됨</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>stride=(1,1), padding=1: </b>kernel의 이동은 세로로 1, 가로로 1만큼. conv층에 들어오는 데이터에 padding을 양옆에 1만큼 해줌</span>

    - <span style = 'font-size:1.1em;line-height:1.5em'><b>2nd MaxPooling</b></span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>kernel_size=(2,2): </b>kernel의 크기는 height 2, width 2. 이러한 kernel이 conv layer의 out_channel개만큼 생성됨</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>stride=(2,2): </b>kernel의 이동은 세로로 2, 가로로 2만큼.</span>
        
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Flatten</b></span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>x.reshape(): </b>FFNN의 입력값으로 넣을 수 있도록, 2nd Pooling Layer의 결과 값인 (batch_size, 128, 7, 7) 크기의 tensor를 (batch_size, 128\*7\*7)크기의 tensor로 형태 변환</span>
        
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>1st FFNN, 2nd FFNN</b></span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>1st FFNN: </b>7\*7\*128차원의 벡터를 100차원의 벡터로 보내는 FFNN (with relu 활성화 함수)</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>2nd FFNN: </b>100차원의 벡터를 10차원(=class 수)의 벡터로 보내는 FFNN (원래는 softmax함수를 함께 써야 하지만, Pytorc)</span>
            - <span style = 'font-size:0.9em;line-height:1.5em'>원래는 softmax함수를 함께 써야 하지만, Pytorch의 cross_entropy함수의 특성상, softmax를 생략</span>

In [2]:
class MyNet(nn.Module):
    def __init__(self, in_channels=1, n_classes=10):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=(1,1), padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.fc1   = nn.Linear(7*7*128, 100, bias=True)
        self.fc2   = nn.Linear(100, n_classes, bias=True)
        self.apply(self._init_weights)
        
    def _init_weights(self, submodule):
        if isinstance(submodule, nn.Conv2d):
            nn.init.xavier_normal_(submodule.weight)
            if submodule.bias is not None:
                submodule.bias.data.fill_(0.01)
        if isinstance(submodule, nn.Linear): # submodule이 nn.Linear에서 생성된 객체(혹은 인스턴스이면)
            nn.init.kaiming_normal_(submodule.weight) #해당 submodule의 weight는 He Initialization으로 초기화
            if submodule.bias is not None:
                submodule.bias.data.fill_(0.01) # 해당 submodule의 bias는 0.01로 초기화
                
    def forward(self, x):
        # (n_data, n_channel, height, width)으로 연산 결과의 크기 표기
        # 1st conv layer
        out = self.conv1(x) # shape: (batch,1,28,28) -> (batch,32,28,28)
        out = F.relu(out) 
        out = self.pool1(out) # (batch,32,28,28) -> (batch,32,14,14)
        
        # 2nd conv layer
        out = self.conv2(out) # (batch,32,14,14) -> (batch,64,14,14)
        out = F.relu(out)
        out = self.pool2(out) # (batch,64,14,14) -> (batch,64,7,7)
        
        # Flatten
        out = out.reshape(-1, 7*7*128)
        
        # 1st FFNN
        out = self.fc1(out)
        out = F.relu(out)
        
        # 2nd FFNN
        out = self.fc2(out)
        return out

## 2. train() 함수

- <span style = 'font-size:1.2em;line-height:1.5em'>`train()`함수는 각 iteration마다 다음과 같이 진행됩니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 1.</b> batch_loader로부터 mini-batch x, y 데이터를 획득하고 원하는 device에 위치시키기</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'> <b>n_data: </b>mini-batch data수, <b>1: </b>channel수(흑백이라서 단일 채널. 칼라 이미지(RGB)는 기본으로 3으로 설정됨) <b>28: </b>Width, <b>28: </b>height</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 2.</b> 지난 batch로부터 계산했던 gradient를 초기화(`zero_grad()`)</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 3.</b> 모델에 batch x를 입력하여 forward propagation</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 4.</b> loss function에 모델이 예측한 각 클래스에 속할 확률(`y_pred_prob`)과 실제 레이블 (`y`)을 넣어서 loss 계산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 5.</b> Backpropagation으로 각 parameter의 gradient를 계산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 6.</b> Gradient Descent로 parameter값 update</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 7.</b> `trn_loss` 변수에 mini-batch loss를 누적해서 합산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 8.</b> 데이터 한 개당 평균 train loss 산출</span>

In [15]:
def train(model, data_loader, optimizer, criterion, device):
    model.train()
    trn_loss = 0
    # 매 mini-batch train data마다 반복
    for i, (x, y) in enumerate(data_loader):
        # 1-(1): 모델에 입력하기 위해서 데이터의 형태 변환
        x = x.to(device) # x.shape: (batch_size,1, 28,28)
        y = y.to(device)
        
        # 1-(2): 기존에 계산된 gradient를 0으로 reset
        my_opt.zero_grad()
        
        # 1-(3): Forward Propagation
        y_pred_prob = model(x)
        
        # 1-(4): Loss Calculation
        loss = criterion(y_pred_prob, y)
        
        # 1-(5): Gradient Calculation(Backprop)
        loss.backward()
        
        # 1-(6): Update parameter
        optimizer.step()
        
        # 1-(7): trn_loss에 mini_batch loss를 누적해서 계산하기
        trn_loss += loss.item()
    
    # Step8. 데이터 한 개당 평균 train loss
    avg_trn_loss = trn_loss / len(data_loader.dataset)
    return avg_trn_loss

## 3. evaluate()함수

- <span style = 'font-size:1.2em;line-height:1.5em'>`evaluate()`함수는 각 iteration마다 다음과 같이 진행됩니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 1.</b> batch_loader로부터 mini-batch x, y 데이터를 획득하고 원하는 device에 위치시키기</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'> <b>n_data: </b>mini-batch data수, <b>1: </b>channel수(흑백이라서 단일 채널. 칼라 이미지(RGB)는 기본으로 3으로 설정됨) <b>28: </b>Width, <b>28: </b>height</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 2.</b> 모델에 batch x를 입력하여 forward propagation</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 3.</b> loss function에 모델이 예측한 각 클래스에 속할 확률(`y_pred_prob`)과 실제 레이블 (`y`)을 넣어서 loss 계산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 4.</b> 모델이 예측하는 레이블을 산출 (with `torch.argmax()`)</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 5.</b> Minibatch의 실제 레이블(`y`)과 예측 레이블(`y_pred_label`)을 누적하여 저장</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 6.</b> `eval_loss` 변수에 mini-batch loss를 누적해서 합산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 7.</b> 데이터 한 개당 평균 evaluation loss와 accuracy 산출</span>

In [16]:
def evaluate(model, data_loader, optimizer, criterion, device):
    model.eval() # 모델을 평가모드로!
    eval_loss = 0
    
    results_pred = []
    results_real = []
    with torch.no_grad(): # evaluate()함수에는 단순 forward propagation만 할 뿐, gradient 계산 필요 X.
        for i, (x, y) in enumerate(data_loader):
            # Step 1. mini-batch에서 x,y 데이터를 얻고, 원하는 device에 위치시키기
            x = x.to(device) # x.shape: [batch_size,28,28] -> [batch_size, 784]
            y = y.to(device)

            # Step 2. Forward Propagation
            y_pred_prob = model(x)

            # Step 3. Loss Calculation
            loss = criterion(y_pred_prob, y)
            
            # Step 4. Predict label
            y_pred_label = torch.argmax(y_pred_prob, dim=1)
            
            # Step 5. Save real and predicte label
            results_pred.extend(y_pred_label.detach().cpu().numpy())
            results_real.extend(y.detach().cpu().numpy())
            
            # Step 6. eval_loss변수에 mini-batch loss를 누적해서 합산
            eval_loss += loss.item()

    # Step 7. 데이터 한 개당 평균 eval_loss와 accuracy구하기
    avg_eval_loss = eval_loss / len(data_loader.dataset)
    results_pred = np.array(results_pred)
    results_real = np.array(results_real)
    accuracy = np.sum(results_pred == results_real) / len(results_real)
    
    return avg_eval_loss, accuracy

## 4. 매 Epoch에 드는 시간 측정

In [17]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## 5. 학습하기

- <span style = 'font-size:1.2em;line-height:1.5em'>Dataset과 Mini-batch를 자동으로 생성할 DataLoader준비하기</span>

In [18]:
# torchvision에서도 MNIST데이터를 제공합니다. 
# 이 데이터를 다운 받을 디렉토리(data_path) 존재 여부를 확인하고 존재하지 않으면 생성 
data_path = 'data'
if not os.path.exists(data_path):
    os.makedirs(data_path)
    
# data 변환 방법 선언 (data transform method)
# 아래 예시: numpy형태의 데이터를 받으면 걔를 tensor로 변환해줘!
transform = transforms.Compose([transforms.ToTensor()])

# dataset을 생성 (torchvision에서 제공하는 데이터를 다운 받고, 위의 방법대로 변환)
trn_dset = datasets.MNIST(root=data_path, train=True, transform=transform, download=True)
tst_dset = datasets.MNIST(root=data_path, train=False, transform=transform, download=True)

- <span style = 'font-size:1.2em;line-height:1.5em'>연산을 수행할 device를 설정하기</span>

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

- <span style = 'font-size:1.2em;line-height:1.5em'>모델에 대한 객체 생성하기</span>

In [20]:
model = MyNet()
model = model.to(device)

- <span style = 'font-size:1.2em;line-height:1.5em'>학습한 모델을 저장할 directory 생성하기</span>

In [21]:
save_dir = 'models'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

- <span style = 'font-size:1.2em;line-height:1.5em'>필요한 hyperparameter값 설정하기</span>

In [22]:
N_EPOCHS = 10
LR = 2e-4
BATCH_SIZE = 2**9

- <span style = 'font-size:1.2em;line-height:1.5em'>Mini-batch를 자동으로 생성할 DataLoader준비하기</span>

In [23]:
trn_loader = DataLoader(trn_dset, batch_size = BATCH_SIZE, shuffle=True, drop_last=False)
tst_loader = DataLoader(tst_dset, batch_size = BATCH_SIZE, shuffle=False, drop_last=False)

- <span style = 'font-size:1.2em;line-height:1.5em'>loss함수 정의하기</span>

In [24]:
loss_func = nn.CrossEntropyLoss(reduction='sum')

- <span style = 'font-size:1.2em;line-height:1.5em'>optimizer 생성하기</span>

In [25]:
my_opt = optim.Adam(model.parameters(), lr = LR)

- <span style = 'font-size:1.2em;line-height:1.5em'>trn_data에 대해서 train()함수를, tst_data에 대해서 evaluate()함수를 반복적으로 호출하면서 모델을 학습</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>매 epoch마다 학습이 마무리되면, 모델 평가를 진행한다</span>

In [26]:
best_val_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    trn_loss = train(model=model, 
                     data_loader=trn_loader, 
                     optimizer=my_opt, 
                     criterion=loss_func,
                     device=device)
    val_loss, accuracy = evaluate(model=model, 
                                  data_loader=tst_loader, 
                                  optimizer=my_opt, 
                                  criterion=loss_func,
                                  device=device)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f'{save_dir}/my_model2.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {trn_loss:.3f} | Test Loss: {val_loss:.3f} | Test Acc: {100*accuracy:.3f}% ')

Epoch: 01 | Time: 1m 36s
	Train Loss: 0.823 | Test Loss: 0.269 | Test Acc: 92.720% 
Epoch: 02 | Time: 1m 37s
	Train Loss: 0.211 | Test Loss: 0.155 | Test Acc: 95.550% 
Epoch: 03 | Time: 1m 36s
	Train Loss: 0.131 | Test Loss: 0.099 | Test Acc: 96.960% 
Epoch: 04 | Time: 1m 37s
	Train Loss: 0.096 | Test Loss: 0.080 | Test Acc: 97.620% 
Epoch: 05 | Time: 1m 37s
	Train Loss: 0.078 | Test Loss: 0.068 | Test Acc: 97.640% 
Epoch: 06 | Time: 1m 37s
	Train Loss: 0.067 | Test Loss: 0.060 | Test Acc: 98.120% 
Epoch: 07 | Time: 1m 37s
	Train Loss: 0.059 | Test Loss: 0.051 | Test Acc: 98.310% 
Epoch: 08 | Time: 1m 37s
	Train Loss: 0.052 | Test Loss: 0.045 | Test Acc: 98.460% 
Epoch: 09 | Time: 1m 37s
	Train Loss: 0.047 | Test Loss: 0.042 | Test Acc: 98.600% 
Epoch: 10 | Time: 1m 37s
	Train Loss: 0.043 | Test Loss: 0.040 | Test Acc: 98.640% 
